##### Lion Optimization Algorithm

##### Import Libraries

In [1]:
import numpy as np
import random 
import matplotlib.pyplot as plt
import math
import cv2
import scipy.special as sp
import copy

In [2]:
def initlion(num_lion,dim,p_high,p_low):
    lion_pos=set() 
    while len(lion_pos)<num_lion:
        lion_pos.add(tuple(p_low+(random.uniform(0, 1)*(p_high-p_low)) for _ in range(dim)))
    return list(lion_pos)

In [3]:
def fitness_function(x):
    return 1/(1+(x[0]**2+x[1]**2+x[2]**2))

In [4]:
def define_normad(index,num):
    normad=[]
    while len(normad)<num:
        k=random.choice(index)
        normad.append(k)
        index.remove(k)
    return normad,index


In [5]:
def define_pride(index,num_prides):
    prides=[]
    for i in range(num_prides):
        prides.append([])
        np=[i for i in range(num_prides)]
    for i in index:
        ind=random.choice(np)
        prides[ind].append(i)
    return prides


In [6]:
def define_gen_pride(prides,s_percent):
    pride=copy.deepcopy(prides)
    for ind, val in enumerate(pride):
        length=math.ceil(s_percent*len(pride[ind]))
        pride_female=[]
        while len(pride_female)<length:
            k=random.choice(pride[ind])
            pride_female.append(k)
            pride[ind].remove(k)
        pride[ind]=[pride_female,pride[ind],[len(pride_female),len(pride[ind])]]
    return pride



In [7]:
def define_gen_normad(normad,s_percent):
    length=math.ceil(s_percent*len(normad))
    normad_male=[]
    while len(normad_male)<length:
        k=random.choice(normad)
        normad_male.append(k)
        normad.remove(k)
    normad=[normad,normad_male,[len(normad),len(normad_male)]]
    return normad

In [8]:
def tournment_point_selection(fitness_val,tournment):
    #print(tournment)
    prob=[]
    fit_sum=0
    prob_sum=0
    for i in tournment:
        fit_sum+=fitness_val[i]
    for i in tournment:
        prob_sum+=fitness_val[i]/fit_sum
        prob.append(prob_sum)
    rand=random.uniform(0,1)
    for val,i in enumerate(prob):
        if i>rand:
            return val

In [9]:
def calculate_R1_R2(female_lion_current, selected_position):
    # Convert positions to numpy arrays for easier calculations
    female_lion_current = np.array(female_lion_current)
    selected_position = np.array(selected_position)

    # Calculate the distance D
    D = np.linalg.norm(selected_position - female_lion_current)

    # Calculate R1
    R1 = selected_position - female_lion_current
    # Normalize R1 to get the direction vector
    R1_normalized = R1 / np.linalg.norm(R1)

    # Generate a random reference vector with at least one component being 1
    reference_vector = [random.choice([0, 1]) for _ in range(len(female_lion_current))]
    while 1 not in reference_vector:
        reference_vector = [random.choice([0, 1]) for _ in range(len(female_lion_current))]
    reference_vector = np.array(reference_vector)
    # print(reference_vector)
    # Calculate R2 (perpendicular to R1)
    R2 = np.cross(R1_normalized, reference_vector)
    return R1.tolist(), R2.tolist(), D

In [10]:
def calculate_new_position(male_lion_pos, selected_area_pos, D):
    male_lion_pos=np.array(male_lion_pos)
    selected_area_pos=np.array(selected_area_pos)
    # Step 1: Calculate the direction vector
    direction_vector = selected_area_pos - male_lion_pos

    # Step 2: Normalize the direction vector
    direction_normalized = direction_vector / np.linalg.norm(direction_vector)

    # Step 3: Generate a random angle between -π/6 and π/6
    angle = random.uniform(-np.pi/6, np.pi/6)

    # Step 4: Rotate the direction vector
    # For 2D: Create a rotation matrix
    cos_angle = np.cos(angle)
    sin_angle = np.sin(angle)

    # Rotation matrix for 2D
    rotation_matrix = np.array([[cos_angle, -sin_angle, cos_angle],
                                 [sin_angle, cos_angle,sin_angle],
                                 [sin_angle, -cos_angle,sin_angle],
                                 ])

    # Rotate the normalized direction vector
    rotated_direction = rotation_matrix @ direction_normalized[:3]  # Keep only the first two dimensions for 2D

    # Step 5: Calculate the new position
    new_position = male_lion_pos + D * rotated_direction

    return new_position.tolist()

In [11]:
def visualize_lion(positions, video_filename='lion_optimization.mp4'):
    fig, ax = plt.subplots(figsize=(10, 7))
    out = cv2.VideoWriter(video_filename, cv2.VideoWriter_fourcc(*'XVID'), 10, (640, 480))

    for gen, pos in enumerate(positions):
        print(pos)
        ax.cla()
        pos_x = [p[0] for p in pos]
        print(pos_x)
        pos_y = [p[1] for p in pos]
        print()
        ax.scatter(pos_x, pos_y, c='blue', label='lion')
        print(gen)
        #ax.scatter(posbest[gen][0], posbest[gen][1], c='red', label='Best cat', marker='*')
        ax.set_title(f'Iteration {gen + 1}')
        ax.set_xlim(-20, 20)
        ax.set_ylim(-20, 20)
        ax.set_xlabel('x')
        ax.set_ylabel('y')
        ax.legend()
        # Save the frame to a video
        fig.canvas.draw()
        img = np.frombuffer(fig.canvas.tostring_rgb(), dtype='uint8')
        img = img.reshape(fig.canvas.get_width_height()[::-1] + (3,))
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        img = cv2.resize(img, (640, 480))  # Resize to fit the video frame
        out.write(img)
    out.release()
    plt.close()

In [14]:
num_lion=100
p_high=-5
p_low=5
dim=3
pos_lion=initlion(num_lion,dim,p_high,p_low)
fitness=[]
max_iter=100
for i in pos_lion:
    fitness.append(fitness_function(i))
normad_rate=0.3
num_prides=3
index=[i for i in range(num_lion)]
s_percent=random.uniform(0.75,0.9)
normad,index=define_normad(index,math.ceil(normad_rate*num_lion))
pride=define_pride(index,num_prides)
pride_gen=define_gen_pride(pride,s_percent)
print(pride_gen)
print(pride)
normad_gen=define_gen_normad(normad,s_percent)
mating_prob=0.4
roaming_prob=0.6
pos_best=pos_lion[fitness.index(max(fitness))]
fit_best=fitness[fitness.index(max(fitness))]
# lambda_val=random.uniform(0.3,0.5)
# new_fitness = fitness[:]
fitnesses=[[fit for fit in fitness]]
positions=[[list(lion) for lion in pos_lion]]
immigration_rate=0.3
# print(pos_best)

[[[3, 0, 8, 58, 14, 99, 28, 50, 82, 18, 26, 96, 43, 17, 2, 47, 36, 38, 53, 90, 16], [27, 37, 61], [21, 3]], [[33, 62, 55, 24, 65, 74, 42, 60, 97, 22, 46, 40, 93, 9, 91, 4, 23, 92, 54, 19, 98], [13, 69, 95], [21, 3]], [[68, 70, 31, 66, 25, 59, 75, 21, 88, 67, 7, 39, 52, 78, 44, 84, 76, 12, 30], [6, 77, 81], [19, 3]]]
[[0, 2, 3, 8, 14, 16, 17, 18, 26, 27, 28, 36, 37, 38, 43, 47, 50, 53, 58, 61, 82, 90, 96, 99], [4, 9, 13, 19, 22, 23, 24, 33, 40, 42, 46, 54, 55, 60, 62, 65, 69, 74, 91, 92, 93, 95, 97, 98], [6, 7, 12, 21, 25, 30, 31, 39, 44, 52, 59, 66, 67, 68, 70, 75, 76, 77, 78, 81, 84, 88]]


In [15]:
iter=0
choice=[i for i in range(2)]
hunter_choice=[i for i in range(3)]
while iter<max_iter:
    print(iter)
    hunter_female_pride=[]
    remaining_female_pride=[]
    male_roaming_pride=[]
    for i,val in enumerate(pride_gen):
        hfp=[]
        lhfp=[]
        rhfp=[]
        chfp=[]
        rfp=[]
        sum=np.zeros((3)).tolist()
        for j in val[0]:
            temp=random.choice(choice)
            if(temp==0):
                hfp.append(j)
                sum=[sum[di]+pos_lion[j][di] for di in range(dim)]
                group=random.choice(hunter_choice)
                if(group==0):
                    lhfp.append(j)
                elif(group==1):
                    rhfp.append(j)
                else:
                    chfp.append(j)
            else:
                rfp.append(j)
        prey=[sum[di]/len(hfp) for di in range(dim)]
########### Pride lions - step 1: hfp lioness
        for ind in hfp:
            newpos=np.zeros((3)).tolist()
            if (ind in lhfp or i in rhfp):
                # print("r,l")
                for di in range(dim):
                    if(((2*prey[di])-pos_lion[ind][di])<prey[di]):
                        newpos[di]=random.uniform(((2*prey[di])-pos_lion[ind][di]),prey[di])
                    else:
                        newpos[di]=random.uniform(prey[di],((2*prey[di])-pos_lion[ind][di]))
                # print(pos_lion[ind])
                # print(newpos)
            else:
                for di in range(dim):
                    if(pos_lion[ind][di]<prey[di]):
                        newpos[di]=random.uniform(pos_lion[ind][di],prey[di])
                    else:
                        newpos[di]=random.uniform(prey[di],pos_lion[ind][di])
                # print("c")
                # print(pos_lion[ind])
                # print(newpos)
            if(fitness_function(newpos)>fitness[ind]):
                pi=((1/fitness_function(newpos))-(1/fitness[ind]))*fitness[ind]
                # print("pi")
                # print((fitness[ind]-fitness_function(newpos)))
                # print(fitness[ind])
                # print(pi)
                rand=random.uniform(0,1)
                # print(rand)
                for di in range(dim):
                    prey[di]=prey[di]+(rand*pi*(prey[di]-newpos[di]))
                    # print((prey[di]-newpos[di]))
                # print("prey")
                # print(prey)
            pos_lion[ind]=newpos
            fitness[ind]=fitness_function(newpos)
        print("ended hfp")

########### Pride lions - step 1: rfp lioness

        tij_count=0
        if(len(fitnesses)==1):
            Tij=2
        else:
            for ind in rfp:
                if(fitness[-1][ind]>fitness[-2][ind]):
                    tij_count+=1
            Tij=max(2,math.ceil(tij_count/2))
        for ind in rfp:
            tournment=set()
            while len(tournment)<Tij:
                temp=random.choice(pride[i])
                tournment.add((temp))
            tour_selection=tournment_point_selection(fitness,pride[i])
            while tour_selection==ind:
                tour_selection=tournment_point_selection(fitness,pride[i])
            # print(ind)
            # print(tour_selection)
            # print(pos_lion[ind])
            # print(pos_lion[tour_selection])
            r1,r2,D=calculate_R1_R2(pos_lion[ind],pos_lion[tour_selection])
            r1 = r1 / np.linalg.norm(r1)
            r2= r2 / np.linalg.norm(r2)
            # print(r1)
            # print(r2)
            # print(D)
            newpos=[pos_lion[ind][di]+(2*D*(random.uniform(0,1))*r1[di])+random.uniform(-1,1)*r2[di]*D for di in range(dim)]
            # print(newpos)
            # print("okay")
            pos_lion[ind]=newpos
            fitness[ind]=fitness_function(newpos)


        roaming_length=math.ceil(len(val[1])*roaming_prob)
        print(val[1])
        for ind in val[1]:
            roaming_selection_area=set()
            best_fit=fitness[ind]
            best_pos=pos_lion[ind]
            while(len(roaming_selection_area)<roaming_length):
                roaming_selection_area.add((random.choice(pride[i])))

            for j in roaming_selection_area:
                d=math.sqrt((pos_lion[ind][0]-pos_lion[j][0])**2
                +(pos_lion[ind][1]-pos_lion[j][1])**2
                +(pos_lion[ind][2]-pos_lion[j][2])**2)
                x=random.uniform(0,2*d)
                new=calculate_new_position(pos_lion[ind], pos_lion[j], x)
                if(fitness_function(new)>best_fit):
                    best_fit=fitness_function(new)
                    best_pos=new
            pos_lion[ind]=best_pos
            fitness[ind]=best_fit
            # print(pos_lion[ind])
        for ind in normad_gen[1]:
            new=[]
            fit=[fitness[di] for di in normad_gen[1]]
            prj=0.1+min(0.5,max(fit)-fitness[ind]/max(fit))
            for j in range(dim):
                rand=random.uniform(0,1)
                if(rand>prj):
                    new.append(pos_lion[ind][j])
                else:
                    new.append(p_low+(random.uniform(0, 1)*(p_high-p_low)))
            if(fitness_function(new)>fitness[ind]):
                pos_lion[ind]=new
                fitness[ind]=fitness_function(new)

        ##### pride - mating
        mating_females=set()
        mating_len=math.ceil(len(val[0])*mating_prob)
        while len(mating_females)<mating_len:
            mating_females.add((random.choice(val[0])))
        # print(len(mating_females))
        for ind in mating_females:
            s=[]
            offspring1=[]
            offspring2=[]
            total_si=0
            for _ in val[1]:
                s_val=random.choice([0,1])
                s.append(s_val)
                if(s_val==1):
                    total_si+=1
            if(total_si==0):
                s[val[1].index(random.choice(val[1]))]=1
                total_si+=1
            for j in range(dim):
                beta=np.random.normal(loc=0.5, scale=1.0)
                offspring1.append(beta*pos_lion[ind][j])
                offspring2.append((1-beta)*pos_lion[ind][j])
                for ind1,k in enumerate(val[1]):
                    # print(pos_lion[k][j])
                    offspring1[j]+=((1-beta)*pos_lion[k][j]*s[ind1])/total_si
                    offspring2[j]+=((beta)*pos_lion[k][j]*s[ind1])/total_si
            pos_lion.append(offspring1)
            fitness.append(fitness_function(offspring1))
            pos_lion.append(offspring2)
            fitness.append(fitness_function(offspring2))
            # print(len(pos_lion))
            if(random.choice([0,1])==1):
                pride_gen[i][0].append(len(pos_lion)-2)
                pride_gen[i][1].append(len(pos_lion)-1)
                
            else:
                pride_gen[i][0].append(len(pos_lion)-1)
                pride_gen[i][1].append(len(pos_lion)-2)
                
        ##### normad - mating
        mating_females_normad=set()
        mating_len=math.ceil(len(val[0])*mating_prob)
        while len(mating_females)<mating_len:
            mating_females.add((random.choice(val[0])))
        for ind in mating_females:
            s=[]
            offspring1=[]
            offspring2=[]
            total_si=0
            fit=[fitness[di] for di in normad_gen[1]]
            best_pos_male_normad=pos_lion[normad_gen[1][fit.index(max(fit))]]
            for j in range(dim):
                beta=np.random.normal(loc=0.5, scale=1.0)
                offspring1.append(beta*pos_lion[ind][j]+((1-beta)*best_pos_male_normad[j]))
                offspring2.append((1-beta)*pos_lion[ind][j]+((beta)*best_pos_male_normad[j]))
            # print(offspring1)
            # print(offspring2)
            pos_lion.append(offspring1)
            fitness.append(fitness_function(offspring1))
            pos_lion.append(offspring2)
            fitness.append(fitness_function(offspring2))
            # print(len(pos_lion))
            if(random.choice([0,1])==1):
                normad_gen[0].append(len(pos_lion)-2)
                normad_gen[1].append(len(pos_lion)-1)
            else:
                normad_gen[0].append(len(pos_lion)-1)
                normad_gen[1].append(len(pos_lion)-2)
        
        ##### female lion immigration
        immigration_len=math.ceil(immigration_rate*pride_gen[i][2][0])
        while(len(pride_gen[i][0])>pride_gen[i][2][0]):
            temp=random.choice(pride_gen[i][0])
            pride_gen[i][0].remove(temp)
            normad_gen[0].append(temp)
        orig_len=len(pride_gen[i][0])
        while(len(pride_gen[i][0])>orig_len-immigration_len):
            temp=random.choice(pride_gen[i][0])
            pride_gen[i][0].remove(temp)
            normad_gen[0].append(temp)
        
        ##### pride to normad
        fit_pride=[fitness_function(pos_lion[di]) for di in pride_gen[i][1]]
        while(len(pride_gen[i][1])>pride_gen[i][2][1]):
            index=fit_pride.index(min(fit_pride))
            fit_pride.remove(fit_pride[index])
            normad_gen[1].append(pride_gen[i][1][index])
            pride_gen[i][1].remove(pride_gen[i][1][index])
        
        # print(len(pos_lion))
    
        ##### noramd to pride
    for i in normad_gen[1]:
        flag=0
        BT=[random.choice([0,1]) for i in range(num_prides)]
        for ind,j in enumerate(BT):
            if j==1:
                for inde,z in enumerate(pride_gen[ind][1]):
                    if(fitness[z]<fitness[i]):
                        pride_gen[ind][1][inde]=i
                        normad_gen[1].remove(i)
                        normad_gen[1].append(z)
                        flag=1
                        break
            if(flag==1):
                break


        ##### remove extra lions
    removal=[]
    fit_val=[fitness[ind] for ind in normad_gen[1]]
    while(len(fit_val)>normad_gen[2][1]):
        inde=fit_val.index(min(fit_val))
        removal.append(normad_gen[1][inde])
        normad_gen[1].pop(inde)
        fit_val.pop(inde)
    num_of_required_fem=[]
    for pr in pride_gen:
        num_of_required_fem.append(pr[2][0]-len(pr[0]))
    # print(num_of_required_fem)
    fit_val=[fitness[ind] for ind in normad_gen[0]]
    choices=[]
    for ind,i in enumerate(num_of_required_fem):
        if i!=0:
            choices.append(ind)
    while(len(choices)>0):
        ind=fit_val.index(max(fit_val))
        rand=random.choice(choices)
        num_of_required_fem[rand]-=1
        if(num_of_required_fem[rand]==0):
            choices.remove(rand)
        pride_gen[rand][0].append(normad_gen[0][ind])
        normad_gen[0].remove(normad_gen[0][ind])
        fit_val.pop(ind)
    while(len(normad_gen[0])>normad_gen[2][0]):
        ind=fit_val.index(min(fit_val))
        removal.append(normad_gen[0][ind])
        normad_gen[0].pop(ind)
        fit_val.pop(ind)
    removal.sort(reverse=True)
    for ind1 in removal:
        pos_lion.pop(ind1)
        fitness.pop(ind1)
    
    # Update pride_gen and normad_gen to reflect this removal
    # Update pride_gen
        for pride in pride_gen:
            pride[0] = [ind if ind < ind1 else ind - 1 for ind in pride[0]]
            pride[1] = [ind if ind < ind1 else ind - 1 for ind in pride[1]]

        # Update normad_gen
        normad_gen[0] = [ind if ind < ind1 else ind - 1 for ind in normad_gen[0]]
        normad_gen[1] = [ind if ind < ind1 else ind - 1 for ind in normad_gen[1]]

    positions.append([list(lion) for lion in pos_lion])
    # Normad 
    iter+=1


0
ended hfp
[27, 37, 61]
ended hfp
[13, 69, 95]
ended hfp
[6, 77, 81]
1
ended hfp
[64, 87, 51]
ended hfp
[40, 3, 70]
ended hfp
[95, 83, 86]
2
ended hfp
[93, 25, 36]
ended hfp
[96, 43, 33]
ended hfp
[8, 12, 92]
3
ended hfp
[45, 99, 79]
ended hfp
[48, 87, 71]
ended hfp
[2, 17, 90]
4
ended hfp
[46, 2, 25]
ended hfp
[83, 88, 77]
ended hfp
[37, 8, 60]
5
ended hfp
[64, 78, 82]
ended hfp
[23, 49, 42]
ended hfp
[46, 71, 2]
6
ended hfp
[43, 50, 77]
ended hfp
[13, 32, 25]
ended hfp
[29, 7, 80]
7
ended hfp
[32, 36, 52]
ended hfp
[12, 25, 19]
ended hfp
[22, 6, 90]
8
ended hfp
[92, 84, 32]
ended hfp
[9, 19, 15]
ended hfp
[18, 87, 82]
9
ended hfp
[55, 50, 95]
ended hfp
[5, 13, 10]
ended hfp
[12, 52, 49]
10
ended hfp
[34, 30, 5]
ended hfp
[2, 8, 82]
ended hfp
[7, 99, 91]
11
ended hfp
[21, 95, 73]
ended hfp
[1, 62, 65]
ended hfp
[43, 6, 48]
12
ended hfp
[13, 99, 57]
ended hfp
[1, 37, 40]
ended hfp
[77, 81, 28]
13
ended hfp
[13, 60, 37]
ended hfp
[1, 24, 26]
ended hfp
[47, 48, 20]
14
ended hfp
[8, 90, 

/var/folders/m9/n7_wwdyx6f978k8xw8r5f6fm0000gn/T/ipykernel_64429/1420193173.py:8: RuntimeWarning: invalid value encountered in divide
  direction_normalized = direction_vector / np.linalg.norm(direction_vector)


In [16]:
visualize_lion(positions)

OpenCV: FFMPEG: tag 0x44495658/'XVID' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


[[0.7794728976479082, 1.7431345041674131, -2.7702583753025856], [1.9024086228388248, 1.038774943588603, -4.4102120710792505], [-2.505626137651104, 4.617386002620125, -1.8961128935327034], [-2.1278687577768585, 2.9272629538581207, -0.7799011257755817], [3.2733446931511923, 4.406784258081801, 3.199208021226209], [2.104406327167508, -0.639179959777251, 3.2619687251747442], [-3.209846821970178, -1.6014193352858372, -2.7043663932211848], [4.330719675049871, 1.854849479614745, 3.5364873044960996], [1.846971791820391, -1.6166496238434753, 1.8324569792661718], [-1.1974858433333786, 1.7678248323042767, 3.6281825694460466], [-0.3158547759294139, 1.2070123061852844, -3.4514212354930436], [3.3247967656162416, 4.231521675442263, -1.6350186054241362], [-2.7723664491567312, -2.3066419271080445, -3.024157339635236], [2.7754900164089946, -4.5212676988990665, -3.4728499709492198], [4.30008128614228, 0.3018094077767408, -1.2568900192221033], [-0.7400589288667989, 2.1580257395920057, 0.5982151566175027], 